In [1]:
import sys
import os
import time
import sqlite3
import json
from pathlib import Path

import numpy as np
import pandas as pd
from ordered_set import OrderedSet as oset

sys.path.append("..")
from filesplitter import db, dv8
from filesplitter.clustering import cluster_dataset
from filesplitter.loading import load_dataset
from filesplitter.validate import validate

In [2]:
DATA_DIR = Path("../data/")
# FILENAME = "activemq-broker/src/main/java/org/apache/activemq/broker/BrokerService.java"
FILENAME  = "activemq-broker/src/main/java/org/apache/activemq/broker/region/policy/PolicyEntry.java"
DS = load_dataset(Path(DATA_DIR, "activemq" + ".db"), FILENAME)

In [3]:
entities_df = DS.entities_df()
entities_df

,parent_id,name,kind,start_row,end_row
id,,,,,
13849,216,LOG,field,50.0,50.0
13850,216,dispatchPolicy,field,51.0,51.0
13851,216,subscriptionRecoveryPolicy,field,52.0,52.0
13852,216,sendAdvisoryIfNoConsumers,field,53.0,53.0
2174,216,sendDuplicateFromStoreToDLQ,field,54.0,54.0
...,...,...,...,...,...
66763,None,activemq-unit-tests/src/test/java/org/apache/a...,file,NaN,NaN
66904,None,activemq-unit-tests/src/test/java/org/apache/a...,file,NaN,NaN
67079,None,activemq-unit-tests/src/test/java/org/apache/a...,file,NaN,NaN


In [4]:
jdeo_entities_df = pd.read_csv("../jdeo/activemq-broker.csv", names=["filename", "name", "kind", "block_name"])

In [5]:
jdeo_entities_df[jdeo_entities_df["filename"] == "/" + FILENAME]

,filename,name,kind,block_name
3880,/activemq-broker/src/main/java/org/apache/acti...,LOG,field,W0
3881,/activemq-broker/src/main/java/org/apache/acti...,dispatchPolicy,field,W1AAAAAAAAAAAA
3882,/activemq-broker/src/main/java/org/apache/acti...,deadLetterStrategy,field,W1AAAAAAAAAAAB
3883,/activemq-broker/src/main/java/org/apache/acti...,memoryLimit,field,W1AAAAAAAAAABA
3884,/activemq-broker/src/main/java/org/apache/acti...,lazyDispatch,field,W1AAAAAAAAAABB
...,...,...,...,...
3954,/activemq-broker/src/main/java/org/apache/acti...,getMessageGroupMapFactory,method,W1BAB
3955,/activemq-broker/src/main/java/org/apache/acti...,messageGroupMapFactory,field,W1BB
3956,/activemq-broker/src/main/java/org/apache/acti...,networkBridgeFilterFactory,field,W2
3957,/activemq-broker/src/main/java/org/apache/acti...,maxDestinations,field,W3


In [6]:
entities_df["block_name"] = "W999"

In [7]:
# entities_df.loc[(entities_df["name"] == "LOG") & (entities_df["kind"] == "field"), "block_name"] = "test"

In [8]:
for _, row in jdeo_entities_df.iterrows():
    jdeo_name = row["name"]
    jdeo_kind = row["kind"]
    jdeo_block_name = row["block_name"]
    entities_df.loc[(entities_df["name"] == jdeo_name) & (entities_df["kind"] == jdeo_kind), "block_name"] = jdeo_block_name

In [9]:
entities_df

,parent_id,name,kind,start_row,end_row,block_name
id,,,,,,
13849,216,LOG,field,50.0,50.0,W0
13850,216,dispatchPolicy,field,51.0,51.0,W1AAAAAAAAAAAA
13851,216,subscriptionRecoveryPolicy,field,52.0,52.0,W1AAAAAAAAB
13852,216,sendAdvisoryIfNoConsumers,field,53.0,53.0,W7AAAAAAAAAAABAAAAAAABAAAAAAAAAAB
2174,216,sendDuplicateFromStoreToDLQ,field,54.0,54.0,W7AAAAAAAAAAABAAAAAAABAAAAAAAAAB
...,...,...,...,...,...,...
66763,None,activemq-unit-tests/src/test/java/org/apache/a...,file,NaN,NaN,W999
66904,None,activemq-unit-tests/src/test/java/org/apache/a...,file,NaN,NaN,W999
67079,None,activemq-unit-tests/src/test/java/org/apache/a...,file,NaN,NaN,W999


In [10]:
targets_df = entities_df.loc[~(entities_df["kind"] == "file")]
deps_df = DS.target_deps_df

In [11]:
print("Writing DSM...")
with open(Path("../jdeo", "jdeo" + ".dsm.json"), "w") as f:
    json.dump(dv8.to_dsm("jdeo", targets_df, deps_df), f)

Writing DSM...


In [12]:
# print("Writing DRH...")
# with open(Path("../jdeo", "jdeo" + ".drh.json"), "w") as f:
#     json.dump(dv8.to_drh("jdeo" + "-drh", targets_df), f)